In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#DOWNLOAD THE DATA
from google.colab import drive
import zipfile

#MOUNT GDRIVE: DOWNLOAD DATA AND FILTERED LABELS
drive.mount('/content/gdrive',force_remount=True)

# UNZIP ZIP
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/CheXpert-v1.0-small.zip', 'r')
zip_ref.extractall()
zip_ref.close()
print("downloaded files")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Uncompressing zip file
downloaded files


In [0]:
#import sys #For Patch lib
#sys.path.append("gdrive/My Drive/patch")

#CONTRASTIVE LOST OLCAK

In [0]:
import os#for CUDA tracking
#from __future__ import print_function, division
import torch
import pandas as pd
from skimage.io import imread
#from skimage import io
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#MODELS
import re
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

class CheXpertDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
              
        super().__init__()

        self.observations_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.list_classes=['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']

    def __len__(self):
        return len(self.observations_frame)

    def __getitem__(self, idx):
        
        img_name = os.path.join(self.observations_frame.iloc[idx, 0])
        image = imread(img_name)
        image = transforms.ToPILImage()(image)

        observations=self.observations_frame.loc[idx,self.list_classes]
        observations = torch.from_numpy(observations.values.astype(np.float32))
        
        #RETURNING IMAGE AND LABEL SEPERATELY FOR TORCH TRANSFORM LIB
      
        if self.transform:     
            image = self.transform(image)
                
        return image,observations
      
      
      
def chexpert_load(csv_file_name,transformation,batch_size):
  
  cheXpert_dataset = CheXpertDataset(csv_file=csv_file_name,
                                     root_dir='not used', transform=transformation)

  dataloader = DataLoader(cheXpert_dataset, batch_size=batch_size,shuffle=True)

  return cheXpert_dataset,dataloader



use_cuda = True
if use_cuda and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    print("CUDA didn't work")
    device = torch.device('cpu')



In [0]:
for i in range(0,3):
  print(i)

0
1
2


In [0]:
import torch
import numpy as np
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.patches as patches4rectangle


class Jigsaw(object):
    def __init__(self, image_batch, path_permutation_set, grid_crop_size=255, patch_crop_size=75, show=False):

        self.bs, _, self.h, self.w = image_batch.shape
        self.show = show  # print cropped images and show where they are 1 time for each batch
        self.image_batch = image_batch
        self.permutation_set = torch.load(path_permutation_set)
        self.grid_crop_size = grid_crop_size
        self.patch_crop_size = patch_crop_size

    def __call__(self):

        patches = torch.Tensor()
        labels = np.empty(self.bs)
        
        for idx, image in enumerate(self.image_batch):

            permuted_patches, label ,shiftcol,shiftrow, cord_patches = self.puzzled_patches(image)

            if idx == 1 and self.show:
                self.show_cropped_patches(image, shiftcol, shiftrow, permuted_patches, cord_patches, label)
                self.show = False

            patches = torch.cat((patches, permuted_patches))
            labels[idx] = label
        
            
        return patches, labels

      
    def puzzled_patches(self, image):
      
        grid, shiftcol, shiftrow = self.random_crop(image,self.grid_crop_size)
        patches, cord_patches = self.get_patches(grid)
        permuted_patches,label= self.permut(patches)
        
        return permuted_patches, label ,shiftcol,shiftrow, cord_patches

    def random_crop(self, image,cropsize):
      _, h ,_ = image.shape
      [shiftrow, shiftcol] = np.random.randint(h-cropsize-1, size=2)
      
      return image[:,shiftrow:shiftrow+cropsize,shiftcol:shiftcol+cropsize] , shiftcol, shiftrow

    def get_patches(self,grid):
      
      #dive a 225x225 grid to 3x3 
      patches = torch.empty(9,self.patch_crop_size, self.patch_crop_size)
      cord_patches= []
      grid_patch_size = int(self.grid_crop_size/3) #85   
     
      patch_n=0
      for i in range(0,3):
        for j in range(0,3):
          #each patch take random crop to prevent edge following
          
          row_start=0+grid_patch_size*i ; row_finit=row_start+grid_patch_size
          col_start=0+grid_patch_size*j ; col_finit=col_start+grid_patch_size
          
          patches[patch_n,:,:] , scol,srow =self.random_crop(grid[:,row_start:row_finit,col_start:col_finit],self.patch_crop_size)
          cord_patches.append((scol,srow))
          patch_n=patch_n+1
      
      return patches , cord_patches
      
    def permut(self,patches):
      set_size,_= self.permutation_set.shape# Nx9
      
      pick = np.random.randint(set_size)# pick = label of the perm
      perm = self.permutation_set[pick]
      
      permuted_patches=patches[perm.long()]
      
      return permuted_patches, pick
           
    
    def show_cropped_patches(self, image, shift_col, shift_row, permuted_patches, cord_patches, label):
      #Preparing        
      image_draw = transforms.ToPILImage()(image)
      perm_set= self.permutation_set[label]

      #font = ImageFont.truetype("arial.ttf", fontsize)
      #Original Image plot
      fig, ax = plt.subplots(1,figsize=(5,5))
      ax.axis("off")
      
      start_col = shift_col
      start_row = shift_row
      
      for i in range(0,4):
        grid_patch_size = int(self.grid_crop_size/3)
        shift= i*grid_patch_size
        
        ax.plot([start_col, start_col+grid_patch_size*3], [start_row+shift,start_row+shift],color='r',linestyle='dashed')
        ax.plot([start_col+shift,start_col+shift], [start_row, start_row+grid_patch_size*3],color='r',linestyle='dashed')

      #print(cord_patches)
      for idx, (srow, scol) in enumerate(cord_patches):
        #print(idx)
        n= perm_set[idx]
        
        if n<3:
           i = 0
        elif n < 6:
          i = 1
        else:
          i=2
          
        j = n %3
        
        rect = patches4rectangle.Rectangle((shift_col +j*grid_patch_size+scol, shift_row + i*grid_patch_size +srow), 64 , 64, linewidth=2,
                                           edgecolor='g', facecolor='none')
        ax.add_patch(rect)

          
      plt.imshow(image_draw, cmap='Greys_r')
        
      #Permuted Grid
      fig, ax = plt.subplots(3,3, sharex='col', sharey='row',figsize=(5,5))
      fig.subplots_adjust(hspace=0,wspace=0)
        
      for idx in np.arange(9):
          
        if idx<3:
          i = 0
        elif idx < 6:
          i = 1
        else:
          i=2
          
        j = idx %3

        patch_show = transforms.ToPILImage()(permuted_patches[idx,:,:])
        ax[i,j].imshow(patch_show, cmap='Greys_r')
        ax[i,j].axis('off')
       
      #Aligned Grid
      fig, ax = plt.subplots(3,3, sharex='col', sharey='row',figsize=(5,5))
      fig.subplots_adjust(hspace=0,wspace=0)
      
      for index,idx in enumerate(perm_set):
        
        idx = int(idx.item())
        
        if idx<3:
          i = 0
        elif idx < 6:
          i = 1
        else:
          i=2
          
        j = idx %3
          
        patch_show = transforms.ToPILImage()(permuted_patches[index,:,:])
        ax[i,j].imshow(patch_show, cmap='Greys_r')
        ax[i,j].axis('off')
            


In [0]:
#HOW TO USE RES BLOCK AT THE END AS A Class there operations between features and classifier that causes dimension conflict
#LOSS FUNC JUST A 8 WAY SOFTMAX

class JigSawHead(torch.nn.Module):
    def __init__(self, D_in, D_out=8):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(RelativePositionHead, self).__init__()
        self.res_block1 = models.resnet.BasicBlock(D_in,512)
        self.res_block2 = models.resnet.BasicBlock(512,512)
        self.classifier = torch.nn.Linear(512,D_out)

    def forward(self, x):
        
        #print(x.shape)
        x = self.res_block1(x)
        x = self.res_block2(x)
        
        _,N,_,_ = x.shape
        # combining two representation (batch is [bs,ch,h,w ])
        x = torch.stack([torch.cat((x[idx,:,:,:],x[idx+1,:,:,:]))
                     for idx in range(0,N,2)], dim=0)
        
        #linear output with 8 outpts(directions)
        y_pred = self.classifier(x)
        return y_pred

      
      
class Basic_JigsawHead(torch.nn.Module):
    def __init__(self, D_in, D_out):
        """
        No task head just concating what comes out just before default classifer then applying linear
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(Basic_JigsawHead, self).__init__()
        self.classifier = torch.nn.Linear(D_in*9,D_out)

    def forward(self, x):        
        N,_ = x.shape
        # combining two representation (batch is [bs,ch,h,w ])
        
        def stack_tiles(tiles):
          tile_stacked = torch.Tensor().cuda()
          for tile in tiles:
            tile_stacked = torch.cat((tile_stacked,tile))
          return tile_stacked
          
        x = torch.stack([stack_tiles(x[idx:idx+9,:]) for idx in range(0,N,9)], dim=0).cuda()
        
        #linear output with 8 outpts(directions)
        y_pred = self.classifier(x)
        return y_pred
    


In [0]:
learning_rate=0.0001
batch_size=16
dtype = torch.float32
resize=320
num_classes=50
grid_crop_size=225
patch_crop_size=64

file_name_p_set = F"permutation_set_{num_classes}.pt"
PATH_p_set = F"gdrive/My Drive/summerthesis/permutation_sets/{file_name_p_set}"

#just ToTensor before pathch
transform_train= transforms.Compose([          transforms.RandomCrop(320),
                                               transforms.ToTensor()])

#after patch transformation
transform_after_jigsaw_patch= transforms.Compose([    transforms.ToPILImage(),
                                               #transforms.Resize(resize),
                                               transforms.ToTensor(),
                                               transforms.Lambda(lambda x: torch.cat([x, x, x], 0)),
                                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])                                 

cheXpert_train_dataset, dataloader = chexpert_load("CheXpert-v1.0-small/train.csv",transform_train,batch_size)



model=models.densenet121(num_classes=num_classes)
#model.classifier=RelativePositionHead(1024,num_classes)
model.classifier=Basic_JigsawHead(1024,num_classes)
model=model.to(device=device)


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion =torch.nn.CrossEntropyLoss().cuda()

num_epochs=5
for epoch in range(num_epochs):
    for i,  (images, observations) in enumerate(dataloader):   # Load a batch of images with its (index, data, class)
        
        
        jigsaw_patcher = Jigsaw(images, PATH_p_set, grid_crop_size=grid_crop_size, patch_crop_size=patch_crop_size, show=False)
        jigsaw_patches, labels =  jigsaw_patcher()
        jigsaw_patches = torch.stack([transform_after_jigsaw_patch(x_i) 
                     for i, x_i in enumerate(torch.unbind(jigsaw_patches, dim=0))], dim=0)
                
          
        jigsaw_patches = jigsaw_patches.to(device=device, dtype=dtype)
        labels = torch.from_numpy(labels)
        labels = labels.long()
        labels = labels.to(device=device, dtype=torch.long)
        
        
        outputs = model(jigsaw_patches)                             # Forward pass: compute the output class given a image
        
        #print("model outputs ",outputs.shape)

        loss = criterion(outputs, labels)           # Compute the loss: difference between the output class and the pre-given label

        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        
        if (i+1) % 50 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(cheXpert_train_dataset)//batch_size, loss))


print('training done')



file_name = F"jigsaw_num_classes{num_classes}_epoch{num_epochs}_batch{batch_size}_grid_size{grid_crop_size}_patch_size{patch_crop_size}.pth"
PATH=F"/content/gdrive/My Drive/summerthesis/saved_model/{file_name}"
torch.save(model.state_dict(), PATH)
print('saved  model to google drive')




Epoch [1/5], Step [50/13963], Loss: 3.1644
Epoch [1/5], Step [100/13963], Loss: 3.1263
Epoch [1/5], Step [150/13963], Loss: 2.4570
Epoch [1/5], Step [200/13963], Loss: 1.8843
Epoch [1/5], Step [250/13963], Loss: 1.3998
Epoch [1/5], Step [300/13963], Loss: 1.3370
Epoch [1/5], Step [350/13963], Loss: 0.8828
Epoch [1/5], Step [400/13963], Loss: 0.6529
Epoch [1/5], Step [450/13963], Loss: 0.9817
Epoch [1/5], Step [500/13963], Loss: 0.4822
Epoch [1/5], Step [550/13963], Loss: 0.5752
Epoch [1/5], Step [600/13963], Loss: 0.3148
Epoch [1/5], Step [650/13963], Loss: 0.2749
Epoch [1/5], Step [700/13963], Loss: 0.4265
Epoch [1/5], Step [750/13963], Loss: 0.2435
Epoch [1/5], Step [800/13963], Loss: 0.1025
Epoch [1/5], Step [850/13963], Loss: 0.1357
Epoch [1/5], Step [900/13963], Loss: 0.0935
Epoch [1/5], Step [950/13963], Loss: 0.1925
Epoch [1/5], Step [1000/13963], Loss: 0.3215
Epoch [1/5], Step [1050/13963], Loss: 0.2519
Epoch [1/5], Step [1100/13963], Loss: 0.3510
Epoch [1/5], Step [1150/13963]